In [ ]:
from utils import read_sql_query

In [ ]:
trn = read_sql_query("select * from stocks.transactions")

In [ ]:
start_date = '2024-01-01'
end_date = '2024-12-31'


bens_direitos = read_sql_query(
    f"""
    WITH last_year_position AS (
        SELECT
            ticker,
            SUM(quantity) AS quantity_old,
            SUM(
                CASE
                WHEN quantity > 0
                THEN price * quantity + COALESCE(taxes, 0)
                ELSE 0
                END
            ) AS total_invested_old,
            -SUM(
                price * quantity + COALESCE(taxes, 0)
            ) AS profit_old
        FROM stocks.transactions
        WHERE date < DATE '{start_date}'
        GROUP BY ticker
    ),
    current_year_position AS (
        SELECT
            ticker,
            SUM(quantity) AS quantity_current,
            SUM(
                CASE
                WHEN quantity > 0
                THEN price * quantity + COALESCE(taxes, 0)
                ELSE 0
                END
            ) AS total_invested_current,
            -SUM(
                price * quantity + COALESCE(taxes, 0)
            ) AS profit_current
        FROM stocks.transactions
        WHERE date <= DATE '{end_date}'
        GROUP BY ticker
    )
    SELECT
        ticker,
        COALESCE(total_invested_old, 0) AS total_invested_old,
        COALESCE(total_invested_current, 0) AS total_invested_current,
        COALESCE(quantity_old, 0) AS quantity_old,
        COALESCE(quantity_current, 0) AS quantity_current,
        COALESCE(profit_old, 0) AS profit_old,
        COALESCE(profit_current, 0) AS profit_current
    FROM last_year_position
    FULL OUTER JOIN current_year_position
    USING(ticker)
    """
)

In [ ]:
for col in ["old", "current"]:
    bens_direitos.loc[
        bens_direitos[f"profit_{col}"] == -bens_direitos[f"total_invested_{col}"],
        f"profit_{col}"
    ] = 0


In [ ]:
bens_direitos = bens_direitos.sort_values(
    ["quantity_current", "quantity_old"],
    ascending=[False, False]
).query("~((total_invested_old == total_invested_current) & (quantity_old	== quantity_current) & (profit_old == profit_current))")

In [ ]:
bens_direitos.reset_index(drop=True).assign(
    discrimination=lambda df: df["ticker"] + " - " + df["quantity_current"].astype(int).astype(str) + " ações emitidas"
)[[
    "ticker", "discrimination", "total_invested_old",
    "total_invested_current", "quantity_old", "quantity_current",
    "profit_old", "profit_current"
]]

In [ ]:
trn.query("ticker == 'JBSS3'")

In [ ]:
avgp = calculate_avg_price(trn)

In [ ]:
from utils import persist_dataframe_to_database

In [ ]:
persist_dataframe_to_database(
    avgp[[
        "date", "ticker", "quantity", "price", "taxes", "avg_price", "current_quantity", "_processed_at"
    ]],
    "stocks", "transactions"
)

In [ ]:
avgp.query("ticker == 'LEVE3'")